In [1]:
import config
from model import Translator
from transformers import AutoTokenizer

src_tokenizer = AutoTokenizer.from_pretrained(config.SRC_MODEL_NAME)
tgt_tokenizer = AutoTokenizer.from_pretrained(config.TGT_MODEL_NAME)

translator = Translator.load_from_checkpoint(
    "../checkpoints/epoch=0-step=16665.ckpt",
    src_tokenizer=src_tokenizer,
    tgt_tokenizer=tgt_tokenizer,
)

/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from dataset import Iwlst2015DataModule

data_module = Iwlst2015DataModule(src_tokenizer, tgt_tokenizer, max_length=128, batch_size=4)
data_module.setup()

Found cached dataset mt_eng_vietnamese (/home/hoang/.cache/huggingface/datasets/mt_eng_vietnamese/iwslt2015-vi-en/1.0.0/53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71)
Found cached dataset mt_eng_vietnamese (/home/hoang/.cache/huggingface/datasets/mt_eng_vietnamese/iwslt2015-vi-en/1.0.0/53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71)
Loading cached processed dataset at /home/hoang/.cache/huggingface/datasets/mt_eng_vietnamese/iwslt2015-vi-en/1.0.0/53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71/cache-9cf68e9f1ede0163.arrow


  0%|          | 0/133318 [00:00<?, ?ex/s]

Loading cached processed dataset at /home/hoang/.cache/huggingface/datasets/mt_eng_vietnamese/iwslt2015-vi-en/1.0.0/53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71/cache-1f0f26e6f8f9de78.arrow


  0%|          | 0/1269 [00:00<?, ?ex/s]

In [4]:
for batch in data_module.train_dataloader():
    print(batch)
    break

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{'src': {'input_ids': tensor([[  101,  1999,  5773,  1010,  2055,  2028,  1999,  2274,  2317,  7206,
          2056,  1010,  1004, 22035,  2102,  1025,  2748,  1010,  2028,  1997,
          1996,  2502,  4436,  2339,  1045,  5444,  2114, 13857,  8112,  2003,
          2138,  2002,  2001,  2019,  3060,  1011,  2137,  1012,  1004, 22035,
          2102

In [14]:
import torch


def translate(text: str):
    self = translator

    src_token_ids, src_attention_mask = src_tokenizer(
        text, return_token_type_ids=False, return_tensors="pt"
    ).values()

    src_token_ids = src_token_ids.to(self.device)
    src_attention_mask = src_attention_mask.to(self.device)

    print(src_token_ids, src_attention_mask)

    tgt_token_ids = torch.tensor([[tgt_tokenizer.cls_token_id]], device=self.device)
    tgt_attention_mask = torch.tensor([[1]], device=self.device)

    for _ in range(config.MAX_SEQ_LEN):
        logits = translator(
            src_token_ids, tgt_token_ids, src_attention_mask, tgt_attention_mask
        )

        next_tgt_token_id = torch.argmax(logits[:, -1, :], keepdim=True, dim=-1)
        tgt_token_ids = torch.cat([tgt_token_ids, next_tgt_token_id], dim=-1)
        tgt_attention_mask = torch.cat(
            [
                tgt_attention_mask,
                torch.ones_like(next_tgt_token_id, dtype=torch.int64)
                if next_tgt_token_id != tgt_tokenizer.pad_token_id
                else torch.zeros_like(next_tgt_token_id, dtype=torch.int64),
            ],
            dim=-1,
        )

        if next_tgt_token_id == tgt_tokenizer.sep_token_id:
            break

    return tgt_tokenizer.decode(tgt_token_ids[0])


In [17]:
translate("I am a student")

tensor([[ 101, 1045, 2572, 1037, 3076,  102]]) tensor([[1, 1, 1, 1, 1, 1]])


'<s> Cảm ơn. </s>'